In [1]:
# -*-coding: utf-8 -*-
# 对txt文件进行中文分词
from utils import files_processing
import jieba
import os


# 源文件所在目录
source_folder = './three_kingdoms/source'
segment_folder = './three_kingdoms/segment'

# 字词分割，对整个文件内容进行字词分割
def segment_lines(file_list,segment_out_dir,stopwords=[]):
    for i,file in enumerate(file_list):
        segment_out_name=os.path.join(segment_out_dir,'segment_{}.txt'.format(i))
        with open(file, 'rb') as f:
            document = f.read()
            document_cut = jieba.cut(document)
            sentence_segment=[]
            for word in document_cut:
                if word not in stopwords:
                    sentence_segment.append(word)
            result = ' '.join(sentence_segment)
            result = result.encode('utf-8')
            with open(segment_out_name, 'wb') as f2:
                f2.write(result)

# 对source中的txt文件进行分词，输出到segment目录中
file_list=files_processing.get_files_list(source_folder, postfix='*.txt')
segment_lines(file_list, segment_folder)


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\admin\AppData\Local\Temp\jieba.cache
Loading model cost 0.805 seconds.
Prefix dict has been built succesfully.


In [4]:
# -*-coding: utf-8 -*-
# 先运行 word_seg进行中文分词，然后再进行word_similarity计算
# 将Word转换成Vec，然后计算相似度 
from gensim.models import word2vec
import multiprocessing

# 如果目录中有多个文件，可以使用PathLineSentences
segment_folder = './three_kingdoms/segment'
sentences = word2vec.PathLineSentences(segment_folder)

# 设置模型参数，进行训练
model = word2vec.Word2Vec(sentences, size=100, window=3, min_count=1)
print(model.wv.similarity('曹操', '刘备'))
print(model.wv.similarity('曹操', '张飞'))
print(model.wv.most_similar(positive=['曹操', '刘备'], negative=['张飞']))
# 设置模型参数，进行训练
model2 = word2vec.Word2Vec(sentences, size=128, window=5, min_count=5, workers=multiprocessing.cpu_count())
# 保存模型
model2.save('./models/word2Vec.model')
print(model2.wv.similarity('曹操', '刘备'))
print(model2.wv.similarity('曹操', '张飞'))
print(model2.wv.most_similar(positive=['曹操', '刘备'], negative=['张飞']))


0.9798291
0.9836436
[('丞相', 0.9929007291793823), ('主公', 0.992485523223877), ('闻将军', 0.9924496412277222), ('某', 0.9917148947715759), ('叫', 0.9911550283432007), ('臣', 0.9910746812820435), ('不然', 0.9902763366699219), ('今', 0.9902098178863525), ('今番', 0.9899737238883972), ('商议', 0.9896661043167114)]
0.84855926
0.8323382
[('臣', 0.9916843771934509), ('丞相', 0.9909281134605408), ('说客', 0.9878779649734497), ('王命', 0.9858132004737854), ('非', 0.9857558012008667), ('陛下', 0.9852398633956909), ('朕', 0.9851703643798828), ('主公', 0.9840683937072754), ('何故', 0.9835489988327026), ('孔明答', 0.9829833507537842)]
